In [ ]:
%matplotlib inline

# Efficient global optimization using a Gaussian process regressor.


In [ ]:
from __future__ import annotations

from gemseo import configure
from gemseo import configure_logger
from gemseo import sample_disciplines
from gemseo.algos.design_space import DesignSpace
from gemseo.disciplines.analytic import AnalyticDiscipline
from gemseo.mlearning.regression.algos.gpr import GaussianProcessRegressor

from gemseo_mlearning.active_learning.active_learning_algo import ActiveLearningAlgo

configure(False, False, True, False, False, False, False)
configure_logger()

We consider the Rosenbrock function $f(x,y)=(1-x)^2+100(y-x^2)^2$:



In [ ]:
discipline = AnalyticDiscipline({"z": "(1-x)**2+100*(y-x**2)**2"}, name="Rosenbrock")

defined over the input space $[-2,2]^2$:



In [ ]:
input_space = DesignSpace()
input_space.add_variable("x", lower_bound=-2, upper_bound=2, value=1.0)
input_space.add_variable("y", lower_bound=-2, upper_bound=2, value=1.0)

First,
we create an initial training dataset using an optimal LHS including 30 samples:



In [ ]:
learning_dataset = sample_disciplines([discipline], input_space, "z", 30, "OT_OPT_LHS")

and an initial Gaussian process regressor:



In [ ]:
gpr = GaussianProcessRegressor(learning_dataset)

Thirdly,
we look for 20 points that will help us get closer to the minimum;
by default,
for this purpose,
the active learning algorithm looks for the point maximizing the expected improvement.



In [ ]:
active_learning = ActiveLearningAlgo("Minimum", input_space, gpr)
active_learning.set_acquisition_algorithm("DIFFERENTIAL_EVOLUTION", max_iter=1000)
# active_learning.set_acquisition_algorithm("fullfact", n_samples=30 * 30)
active_learning.acquire_new_points(discipline, 20)

Lastly,
we plot the training points,
the original model,
the Gaussian process regressor
and the expected improvement
after the last acquisition:



In [ ]:
active_learning.plot_acquisition_view(discipline=discipline)